In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from HelperFunctions import *
from PtOccupancyFunctions import *
import scipy.integrate
import pprint
from FindDataFunctions import *

In [15]:
# occ_path = "/home/helge/Applications/matsim/matsim-bimodal.git/master/scenarios/fine_grid/unimodal/output/multi/1000reqs/20drt/unimodal/ITERS/it.0/0.drt_occupancy_time_profiles_drt.txt"
# detour_path = "/home/helge/Applications/matsim/matsim-bimodal.git/master/scenarios/fine_grid/unimodal/output/multi/1000reqs/20drt/unimodal/ITERS/it.0/0.drt_detours_drt.csv"
occ_path = "/home/helge/Applications/matsim/matsim-bimodal.git/master/scenarios/fine_grid/unimodal/output/multi/2000reqs/20drt/unimodal/ITERS/it.0/0.drt_occupancy_time_profiles_drt.txt"
detour_path = "/home/helge/Applications/matsim/matsim-bimodal.git/master/scenarios/fine_grid/unimodal/output/multi/2000reqs/20drt/unimodal/ITERS/it.0/0.drt_detours_drt.csv"
paths = {"drt_occupancy": occ_path, "drt_detours": detour_path}

occ,_ = getDrtOccupandyAndStandingFrac(paths, False, True)
det = getDrtDetours(paths).mean()

print(occ)

1.0232166666666667


In [2]:
# path = "/home/helge/Applications/matsim/matsim-bimodal/scenarios/fine_grid/bimodal/output"
# path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/SmallLinkRouting/UniformDist/Zeta1/"
# path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/SmallLinkRouting/TaxiDataDist1250New200GridSpacing"
# path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/SmallLinkRouting/TaxiDataDist1250/"
# path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/LongLinkRouting/VaryE/Uniform2000Mean100Spacing/"
path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/LongLinkRouting/VaryE/Uniform2000Mean100SpacingToLBy2/"

In [3]:
bim_dirs = getBimDirs(path, 800)
uni_dirs = getUniDirs(path, 800)
car_dir = getCarDir(path)

pprint.pprint(bim_dirs)
print('\n---\n')
pprint.pprint(uni_dirs)
print('\n---\n')
pprint.pprint(car_dir)

[{'drt_detours': '/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/LongLinkRouting/VaryE/Uniform2000Mean100SpacingToLBy2/800drt/l_200/bimodal/ITERS/it.0/0.drt_detours_drt.csv',
  'drt_dists': '/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/LongLinkRouting/VaryE/Uniform2000Mean100SpacingToLBy2/800drt/l_200/bimodal/ITERS/it.0/0.vehicleDistanceStats_drt.csv',
  'drt_occupancy': '/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/LongLinkRouting/VaryE/Uniform2000Mean100SpacingToLBy2/800drt/l_200/bimodal/ITERS/it.0/0.drt_occupancy_time_profiles_drt.txt',
  'drt_trips': '/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/LongLinkRouting/VaryE/Uniform2000Mean100SpacingToLBy2/800drt/l_200/bimodal/ITERS/it.0/0.drt_trips_drt.csv',
  'ph_modestats

In [6]:
ell_list = np.empty(20)
cummulative_drt_bimodal = np.empty(20)
cummulative_train_bimodal = np.empty(20)
cummulative_car_bimodal = np.empty(20)
av_drt_occs = np.empty(20)
av_drt_occs_non_empty = np.empty(20)
av_drt_occs_non_standing = np.empty(20)
wait_times = np.empty(20)
wait_times_served_only = np.empty(20)
wait_times_drt_unimodal_legs = np.empty(20)
wait_times_drt_bimodal_legs = np.empty(20)
serviced_fracts = np.empty(20)
av_pt_occs = np.empty(20)
bi_or_unimodal = np.empty((20,2))
drt_pt_person_km = np.empty((20,2))
av_detours = np.empty(20)
drt_wait_times = np.empty(20)
pt_wait_times = np.empty(20)
tt_ratios_beyond_lcut = np.empty(20)
tt_bimodal = np.empty(20)
# rt_bimodal = np.empty(20)
standing_fractions = np.empty(20)
ph_modestats = np.empty((20,5))
pkm_modestats = np.empty((20,3))

ph_columns=["drt_travel", "drt_wait", "pt_travel", "pt_wait", "walk_travel"]
pkm_columns=["drt", "pt", "walk"]

In [7]:
# For sorting out unsuccessfull car trips
car_unsuccess_idx = getTripSuccess(car_dir)["tripSuccess"]
car_unsuccess_idx = car_unsuccess_idx[~car_unsuccess_idx].index
trips_car = getTrips(car_dir, "car").drop(car_unsuccess_idx, errors='ignore')
dists_car = trips_car["traveled_distance"]
cummulative_car = dists_car.sum()
tt_car = trips_car["trav_time"].mean()

trips_unimodal = getTrips(uni_dirs, "unimodal")
trips_unimodal.drop(car_unsuccess_idx, axis=0, errors='ignore')
tt_unimodal = trips_unimodal["trav_time"].mean()
wait_time_unimodal = trips_unimodal["wait_time"].mean()
# ride_time_unimodal = (trips_unimodal["trav_time"] - trips_unimodal["wait_time"]).mean()

for i,dic in enumerate(bim_dirs):
    # drt/pt_person_km can be get easier
    ell_list[i] = getEll(dic["root"])
    print("Ell: ", ell_list[i])
    trips = getTrips(dic, "bimodal").drop(car_unsuccess_idx, errors='ignore')
    drt_trips = getDrtTrips(dic).drop(car_unsuccess_idx, errors='ignore')
    drt_veh_dists = getDrtVehicleDistances(dic)
    trip_succ = getTripSuccess(dic).drop(car_unsuccess_idx, errors='ignore')
    av_drt_occs_non_standing[i], standing_frac = getDrtOccupandyAndStandingFrac(dic, False)
    av_drt_occs_non_empty[i], _ = getDrtOccupandyAndStandingFrac(dic, True)
    av_drt_occs[i], _ = getDrtOccupandyAndStandingFrac(dic, False, True)
#     av_drt_occs_non_empty[i] = drt_occupancy_non_empty
#     av_drt_occs_non_standing[i] = drt_occupancy_non_standing
    av_pt_occs[i], pt_occ_variance = getPtOccupancy(dic)
    drt_detours = getDrtDetours(dic)
    pt_dist = getCummulativePtDistance(dic)
    ph_modestats[i], pkm_modestats[i] = getModeStats(dic, ph_columns, pkm_columns)
    
    drt_pt_person_km[i,0] = drt_trips["travelDistance_m"].sum()
    drt_pt_person_km[i,1] = trips["traveled_distance"].sum()-drt_pt_person_km[i,0]
    serviced_fracts[i] = len(trip_succ[trip_succ["tripSuccess"]]) / len(trip_succ)
#     display(trip_succ[~trip_succ["tripSuccess"]])
    print("Servability: ", serviced_fracts[i])
#     display(len(dists_car.sort_index()))
#     display(trip_succ)
    dists_car_for_rejected = dists_car.loc[
        trip_succ[~trip_succ["tripSuccess"]].index.to_numpy()
    ]
    cummulative_drt_bimodal[i] = drt_veh_dists.sum()
    cummulative_car_bimodal[i] = dists_car_for_rejected.sum()
    cummulative_train_bimodal[i] = pt_dist
    standing_fractions[i] = standing_frac
#     av_trav_time_bimodal = trav_times[pt_in_trips]
    pt_in_trips = trips["modes"].str.contains("pt")
    pt_in_trips_idx = pt_in_trips[pt_in_trips].index
    pt_not_in_trips_idx = pt_in_trips[~pt_in_trips].index
    count_pt_in_trips = pt_in_trips.value_counts()
    bi_or_unimodal[i,0] = count_pt_in_trips[False]
    bi_or_unimodal[i,1] = count_pt_in_trips[True]
    av_detours[i] = drt_detours.mean()
    tt_bimodal[i] = trips["trav_time"].mean()
#     rt_bimodal[i] = (trips["trav_time"] - trips["wait_time"]).mean()
    wait_times[i] = trips["wait_time"].mean()
    
    idx_served = trip_succ[trip_succ["tripSuccess"]].index
    wait_times_served_only[i] = trips.loc[idx_served, "wait_time"].mean()
    
    drt_wait_times[i] = drt_trips["waitTime"].mean()
#     wait_times_drt_unimodal_legs[i] = drt_trips.loc[~pt_in_trips,"waitTime"].mean()
#     wait_times_drt_bimodal_legs[i] = drt_trips.loc[pt_in_trips,"waitTime"].mean()
    wait_times_drt_unimodal_legs[i] = drt_trips.drop(pt_in_trips_idx, errors='ignore')["waitTime"].mean()
    wait_times_drt_bimodal_legs[i] = drt_trips.drop(pt_not_in_trips_idx, errors='ignore')["waitTime"].mean()
    pt_wait_series = (trips["wait_time"] - drt_trips["waitTime"]).fillna(0)
    pt_wait_times[i] = pt_wait_series[pt_wait_series != 0].mean()
    idx_pt_in_trips = pt_in_trips[pt_in_trips].index
    tt_ratios_beyond_lcut[i] = trips["trav_time"][idx_pt_in_trips].sum()/trips_car["trav_time"][idx_pt_in_trips].sum()
    print("-----")

bi_or_unimodal = pd.DataFrame(bi_or_unimodal, index=ell_list, columns=["Unimodal","Bimodal"])
drt_pt_person_km = pd.DataFrame(drt_pt_person_km, index=ell_list, columns=["DRT","PT"])
pkm_modestats = pd.DataFrame(pkm_modestats, index=ell_list, columns=pkm_columns)
ph_modestats = pd.DataFrame(ph_modestats, index=ell_list, columns=ph_columns)

Ell:  200.0
Servability:  1.0
-----


In [9]:
test = np.arange(5)
print(test[0:])

[0 1 2 3 4]


In [8]:
print(av_drt_occs[0])

0.12442722222222224


---
## Theoretical prediction

In [ ]:
nu = 1/10 # per hour
L = 1 #10km
# E = 5e4 #per 100km^2
# E = 8e4 #per 100km^2
E = 125000 #per 100km^2
# E = 500000 #per 100km^2
R0 = 800/130
# mu = 2.5 # per hour
mu = 4 # per hour
#nuELsquared = 1e5/(24*3600)
# delta = 1.5
# b = 2
zeta = 1
# U = 35
beta = 1/6*(np.sqrt(2)+np.log(1+np.sqrt(2)))
# f = 0.85
mean_distance=L/5
v_drt = 30/10
B = 1000/(L*L) # = 300
A = 2*mean_distance
print("Q: ", mu/(nu*E*mean_distance**2))

x = ell_list/(10000) # Factor 10000, because units in simulation are meters
# ell_list = np.linspace(ell_min, ell_max, M)
#f = np.linspace(0,1,N) use f from simulations
f = serviced_fracts

In [ ]:
def inverseGammaDistUnnormalized(x, d_mean=mean_distance, k=3.1):
    if (x==0):
        return 0
    elif (x>L/2):
        return 0
    else:
        z = x/d_mean
        return (z**(-k))*np.exp(-(k-2)/z)


def uniformDist(x):
    A = mean_distance*2
    if (x>A):
        return 0
    else:
        return 1/A
    
normalization_uniform, _ = scipy.integrate.quad(uniformDist,0,np.infty)
assert np.isclose(1, normalization_uniform), "Should already be normalized"
normalization_inverse_gamma, _ = scipy.integrate.quad(inverseGammaDistUnnormalized,0,np.infty)

print("Normalization factor for Uniform distribution: ", normalization_uniform)
print("Normalization factor for InverseGamma distribution: ", normalization_inverse_gamma)

def inverseGammaDist(x):
    return inverseGammaDistUnnormalized(x)/normalization_inverse_gamma

In [ ]:
# testx = np.linspace(0,1,1000)

# plt.figure()
# plt.plot(testx, np.vectorize(probabilityDist)(testx))
# plt.plot(testx, np.vectorize(rpr)(testx))
# plt.show()

In [ ]:
probabilityDist = uniformDist

def getDeltaAvTheory(delta_max, b, mode='mft'):
    delta_max = 1.5
    delta_bar = 2 * delta_max / 3 + 1 / (3 * delta_max)
    if mode=='mft':
        return np.exp(np.log(delta_bar) / np.log(2) * np.log((2*b - 1)))
    elif mode=='bimodal_test':
        return np.exp(np.log(delta_bar) / np.log(2) * np.log((b + 1) / 2))

def cummulative_prob(x):
    return scipy.integrate.quad(probabilityDist,0,x)[0]

# Function for r*p(r)
rpr = lambda r : r*probabilityDist(r)

def size(x,dx):
    return probabilityDist(x)*np.arccos(dx/x)/(2*np.pi)

def tramOccupancy(ell, mode='av'):
    mu_k = 0
    if mode=='int':
        for i in range(0,100):
            dx = ell*i
            mu_k = mu_k + 2*scipy.integrate.quad(size,np.maximum(i*ell,zeta*ell),np.infty,args=(dx))[0]
        mu_k = mu_k - (1-cummulative_prob(zeta*ell))/4
        mu_k = mu_k*nu*E*ell*ell
    elif mode=='av':
#         norm = scipy.integrate.quad(probabilityDist,zeta*ell,np.infty)[0]
#         if probabilityDist == uniformDist and np.isclose(zeta*ell,mean_distance*2):
#             int_r_zetaEll_infty = mean_distance*2
        if probabilityDist == uniformDist and zeta*ell >= mean_distance*2:
            return 0
        else:
#             average_pt_dist = scipy.integrate.quad(lambda x: x*probabilityDist(x)/norm,zeta*ell,np.infty)[0]
            int_r_zetaEll_infty = scipy.integrate.quad(rpr,zeta*ell,np.infty)[0]
        mu_k = int_r_zetaEll_infty/np.pi*E*nu*ell
    
    return mu_k/mu

def getCarbonIndex(ell, f, b, delta, R0):
    dcar = nu*E*L*L*scipy.integrate.quad(rpr,0,np.infty)[0]
    dbus = ((scipy.integrate.quad(rpr,0,zeta*ell))[0] + 2*beta*ell*(1-cummulative_prob(zeta*ell)))*nu*E*L*L
#         dbus = ((zeta*ell)**2/(2*A)+(A-zeta*ell)/A*2*beta*ell)*nu*E*L*L
    dtram = 4*R0*mu*L*L/ell#np.ceil(L/ell)
    Ci_drt = ((delta/b)*f)*dbus/dcar
    Ci_pt = dtram/dcar
    Ci_car = (1-f)*scipy.integrate.quad(rpr,0,np.infty)[0]*nu*E*L*L/dcar
    
    Ci = Ci_drt + Ci_pt + Ci_car
        
    return Ci, Ci_pt, Ci_drt, Ci_car, dcar

def getTramOccupancies(ells, f):
    result = np.empty((len(ells),2))
    for i,ell in enumerate(ells):
        result[i,0] = tramOccupancy(ell)*f[i]
        result[i,1] = tramOccupancy(ell, mode='av')*f[i]
        # For uniform dist
#         A = L/2
#         n_bigger_zeta_ell_Fbar = 1/(2*ell*A)*(A**2-(zeta*ell)**2)
#         print("μ = ", nu*E*ell*ell*n_bigger_zeta_ell_Fbar/(result[i,1]*np.pi))
    
    return result

def getDrtNonEmptyOccupancy(ell, v_drt, B, detours):
#     a = scipy.integrate.quad(rpr,0,zeta*ell)[0]
    a = zeta**2*ell**2/(2*A)
#     b = 2*beta*ell*scipy.integrate.quad(probabilityDist, zeta*ell, np.infty)[0]
    b = 2*beta*ell*(A-zeta*ell)/A

    return nu*E/(v_drt*B)*detours*(a + b)

getCarbonIndexVectorized = np.vectorize(getCarbonIndex)

# av_drt_occs_non_standing = 2
delta_av_theory = getDeltaAvTheory(1.5,av_drt_occs_non_standing, mode='mft')

Ci_theory_mft, Ci_theory_pt_mft, Ci_theory_drt_mft, Ci_theory_car_mft, d_car = \
    getCarbonIndexVectorized(x, f, av_drt_occs_non_standing, delta_av_theory, R0)

Ci_theory, Ci_theory_pt, Ci_theory_drt, Ci_theory_car, _= \
    getCarbonIndexVectorized(x, f, av_drt_occs_non_standing, av_detours, R0)

tram_occs_theory = getTramOccupancies(x, f)#[1 for _ in range(len(x))])

occupancy_theory = np.vectorize(getDrtNonEmptyOccupancy)(x, v_drt, 1000, av_detours)
# occupancy_theory = np.vectorize(getDrtNonEmptyOccupancy)(x, v_drt, 300, delta_av_theory)

---
# Visualization

In [ ]:
%matplotlib notebook
save_figs = False

fig, ax = plt.subplots()
xlabel = r"$\ell/\langle r\rangle$"
x_rescaled = x/mean_distance
print(x)
# Ci_old = Ci

Ci_data_drt = cummulative_drt_bimodal/cummulative_car
Ci_data_car = cummulative_car_bimodal/cummulative_car
Ci_data_pt = R0*cummulative_train_bimodal/cummulative_car
Ci_data =  Ci_data_drt + Ci_data_car + Ci_data_pt

ax.plot(x_rescaled, Ci_data, 'o-', label="Simulation (R0 ≈ {})".format(round(R0,1)), color='#1462e0')
# ax.plot(x_rescaled, Ci_data_drt, 'o-', color='orange', label="Simulation drt contribution")
# ax.plot(x_rescaled, Ci_data_pt, 'o-', color='green', label="Simulation pt contribution")
# ax.plot(x_rescaled, Ci_data_car, 'o-', color='coral', label="Simulation car contribution")

#     print(np.argmin(Ci_data))
print("Min of C_I with R0=", round(R0,2), "at: ", x[np.argmin(Ci_data)], "value: ", np.min(Ci_data))

ax.plot(x_rescaled, Ci_theory, "o--", c='r', label="Theory (R0 ≈ {})".format(round(R0,1)))
# ax.plot(x_rescaled, Ci_theory_drt, "o--", c='orange', label="Theory drt contribution")
# ax.plot(x_rescaled, Ci_theory_pt, "o--", c='green', label="Theory pt contribution")
# ax.plot(x_rescaled, Ci_theory_car, "o--", c='coral', label="Theory car contribution")
ax.set_ylabel(r"$C_I$")
# print(Ci_data/Ci)
# print(ell_list[np.argmin(Ci)])
ax.set_xlabel(xlabel)
ax.legend()
ax.grid()

ax.set_ylim((0,None))

if save_figs:
    fig.savefig("CarbonIdxVaryingl.png", dpi=200)

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('viridis')

Ci_data_drt = cummulative_drt_bimodal/cummulative_car
Ci_data_car = cummulative_car_bimodal/cummulative_car
for R0 in [4, 7]:
    Ci_data_pt = R0*cummulative_train_bimodal/cummulative_car
    Ci_data = Ci_data_drt+Ci_data_car+Ci_data_pt
    Ci_theory_mft, Ci_theory_pt, Ci_theory_drt, Ci_theory_car,_ = getCarbonIndexVectorized(
        x, f, av_drt_occs_non_standing, delta_av_theory, R0
    )
    Ci_theory,_,_,_,_ = getCarbonIndexVectorized(
        x, f, av_drt_occs_non_standing, av_detours, R0
    )
    Ci_data = (
        cummulative_car_bimodal
        + cummulative_drt_bimodal
        + R0 * cummulative_train_bimodal
    ) / cummulative_car
    ax.plot(x_rescaled, Ci_data, "o-", label="Simulation"+r"($\epsilon_0$ ≈ {})".format(round(R0, 1)), color=cmap(R0/7))
    ax.plot(x_rescaled, Ci_theory, "o--", label="Theory"+r"($\epsilon_0$ ≈ {})".format(round(R0,1)), color=cmap(R0/7))
#     ax.plot(x_rescaled, Ci_theory_mft, "o", ls='dotted',label="Theory (R0 ≈ {})".format(round(R0,1)), color=cmap(R0/7))
#     ax.axvline(x_rescaled[np.nanargmin(Ci_data)], color=cmap(R0/7), ls=(0, (1, 1)))
#     ax.axvline(x_rescaled[np.nanargmin(Ci_theory)], color=cmap(R0/7), ls=(0, (3, 10, 1, 10)))
    print("Simulation min of C_I with R0=", round(R0,2), "at: ", x_rescaled[np.argmin(Ci_data)])
    print("Theory min of C_I with R0=", round(R0,2), "at: ", x_rescaled[np.argmin(Ci_theory)])

# newx = np.linspace(x[-1], 1, 10)
# newCi_theory, _, _, _ = getCarbonIndex(newx, [f[-1] for _ in newx], [av_drt_occs_non_standing[-1] for _ in newx], [av_detours[-1] for _ in newx], 7)
# print(newx/D)
# print(Ci_theory)
# ax.plot(newx/D, newCi_theory, "o--")


ax.set_xlabel(xlabel)
ax.grid()
ax.legend()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_ylabel(r"$C_I$")

ax.set_ylim((0,None))
# print(Ci_data)

# save_figs=True
if save_figs:
    fig.savefig("CarbonIdxVaryinglAndR0SimTheo.png", dpi=200)
#     fig.savefig("CarbonIdxVaryinglAndR0SimTheo.pdf")

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

# bi_or_unimodal.set_index(round(1/bi_or_unimodal.reset_index()["index"]), inplace=True)
bi_or_unimodal.set_index(np.round(x_rescaled, decimals=2), inplace=True)
bi_or_unimodal.plot.bar(ax = ax, xlabel=xlabel, ylabel="frequency")

ax.set_title("Travel Mode Share")
ax.grid(axis="y")

if save_figs:
    fig.savefig("BiOrUnimodalBarPlot.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

# bi_or_unimodal.set_index(round(1/bi_or_unimodal.reset_index()["index"]), inplace=True)
drt_pt_person_km.set_index(np.round(x_rescaled, decimals=2), inplace=True)
drt_pt_person_km.plot.bar(ax = ax, xlabel=xlabel, ylabel="meter travelled")

patches, labels = ax.get_legend_handles_labels()

ax.legend(patches, labels)#, frameon=False, bbox_to_anchor=(1.05, 1))
# display(drt_pt_person_km)

ax.set_title("Meters Travelled")
ax.grid(axis="y")

# save_figs = True
if save_figs:
    fig.savefig("PersonKmPerTravelMode.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, serviced_fracts, 'o-')
ax.set_ylabel("f")
ax.set_xlabel(xlabel)

ax.set_ylim((0,1.1))
ax.grid()

if save_figs:
    fig.savefig("ServicedFraction.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

def getWaitTimes(mu_s, link_tt, bi_uni_share, p_drt):
    notp_d = 1-p_d
    ptwait = 1/mu_s
    drtwait = link_tt*sum([i*notp_d**(2*i*(i+1)) for i in range(1,100)])
    twaits = bi_uni_share['Bimodal'] * (ptwait + 2*drtwait) + bi_uni_share['Unimodal'] * drtwait
    return twaits, drtwait, ptwait
    
bi_uni_share = bi_or_unimodal.divide(bi_or_unimodal.sum(axis=1), axis=0)
mu_s = mu/(24*3600)
link_tt = 200/(30/3.6)
p_d = 300/10000
totalwait, drtwait, ptwait = getWaitTimes(mu_s, link_tt, bi_uni_share, p_d)


ax.plot(x_rescaled, wait_times, 'o-', label="Average waiting time (s)")
# ax.plot(x_rescaled, totalwait, 'o--')
ax.set_ylabel("t (s)")
ax.set_xlabel(xlabel)
# ax.set_ylim((0,wait_times.max()*1.1))
ax.legend()
ax.grid()
ax.set_ylim(0,None)

if save_figs:
    fig.savefig("AverageWaitingTime.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, wait_times_served_only, 'o-', label="Average (only served) waiting time (s)")
# ax.plot(x_rescaled, totalwait, 'o--')
ax.set_ylabel("t (s)")
ax.set_xlabel(xlabel)
ax.set_ylim((0,wait_times.max()*1.1))
ax.grid()
ax.legend()

save_figs=False
if save_figs:
    fig.savefig("AverageWaitingTimeServed.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

# test = (av_drt_occs - av_drt_occs.min()) /(av_drt_occs.max() - av_drt_occs.min())
# print(test)
ax.plot(x_rescaled, drt_wait_times, '-o', label='Waiting Time Drt')
ax.plot(x_rescaled, wait_times_drt_unimodal_legs, 'o--', label="Waiting time unimodal legs (s)")
ax.plot(x_rescaled, wait_times_drt_bimodal_legs, 'o--', label="Waiting time bimodal legs (s)")
ax.plot(x_rescaled, 2*wait_times_drt_unimodal_legs, 'o:', label="2 * Waiting time unimodal legs (s)")
# ax.plot(x_rescaled, [drtwait for _ in x], '--o')
ax.set_ylabel("t (s)")
ax.set_xlabel(xlabel)
ax.set_ylim(0,None)
ax.legend(loc='upper left')
ax.grid()

if save_figs:
    fig.savefig("DrtWaitingTime.png", dpi=200)

plt.show()

In [ ]:
fig, ax = plt.subplots()

# test = (av_drt_occs - av_drt_occs.min()) /(av_drt_occs.max() - av_drt_occs.min())
# print(test)
ptwait = 1/(mu/3600)
# Pt wait times are only averaged over people who use pt so wait_times != pt_wait_times + drt_wait_times
ax.plot(x_rescaled, pt_wait_times, '-o', label='Average Waiting Time PT')
ax.plot(x_rescaled, [ptwait for _ in x], '--o', label=r'$1/\mu$')
ax.set_ylabel("t (s)")
ax.set_xlabel(xlabel)
ax.set_ylim(0,ptwait*1.1)
ax.legend()
ax.grid()

if save_figs:
    fig.savefig("PtWaitingTime.png", dpi=200)

plt.show()

In [ ]:
fig, ax = plt.subplots()
#TODO fix this

ax.plot(x_rescaled, av_drt_occs, 'o-')
ax.plot(x_rescaled, occupancy_theory, 'o--')
ax.set_ylabel(r"$\langle b_\mathrm{drt}\rangle$")
ax.set_xlabel(xlabel)
ax.set_ylim(0,None)
ax.grid()

if save_figs:
    fig.savefig("AverageOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()
#TODO fix this

ax.plot(x_rescaled, av_drt_occs_non_empty, 'o-')
ax.set_ylabel(r"$\langle b_\mathrm{drt}^*\rangle$")
ax.set_xlabel(xlabel)
ax.grid()

if save_figs:
    fig.savefig("AverageNonEmptyOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, av_drt_occs_non_standing, 'o-')
ax.set_ylabel(r"$\langle b_\mathrm{drt}\rangle$")
ax.set_xlabel(xlabel)
ax.grid()

if save_figs:
    fig.savefig("AverageNonStandingOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

# Difference to theory for InvGamma comes from periodic BC

ax.plot(x_rescaled, av_pt_occs, 'o-', label="Simulation")

# ax.plot(ell_list, tram_occs_theory[:,1], "o--", c='r', label="Theory computed by"+r" $\langle n\rangle$")
# ax.plot(ell_list, tram_occs_theory[:,0], "o--", c='g', label="Theory computed by"+r" $\tilde{I}$")
ax.plot(x_rescaled, tram_occs_theory[:,1], "o--", c='g', label="Theory")
ax.legend()
ax.grid()

ax.set_ylabel(r"$\langle b_\mathrm{pt}\rangle$")
ax.set_xlabel(xlabel)
ax.set_xlim(0,None)
ax.set_ylim(0,None)

if save_figs:
    fig.savefig("AveragePtOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, av_detours, 'o-')
ax.set_ylabel(r"$\langle \delta\rangle$")
ax.set_xlabel(xlabel)

ax.set_ylim((0,av_detours.max()*1.1))
ax.grid()

save_figs=True
if save_figs:
    fig.savefig("AverageDrtDetour.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, av_drt_occs_non_empty/av_detours, 'o-')
ax.set_ylabel(r"$\langle b^*\rangle/\langle\delta\rangle$")
ax.set_xlabel(xlabel)

ax.set_ylim((0,None))
ax.grid()

if save_figs:
    fig.savefig("BOverDelta.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, tt_bimodal, 'o-', label='bimodal')
ax.plot(x_rescaled, [tt_car for _ in x_rescaled], '--', label='car')
ax.plot(x_rescaled, [tt_unimodal for _ in x_rescaled], '--', label='unimodal')
ax.set_ylabel(r"$t$ (s)")
ax.set_xlabel(xlabel)
ax.legend()
ax.set_ylim(0,1.1*tt_bimodal.max())
ax.grid()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# print(tt_unimodal/tt_car)
# print(tt_bimodal/tt_car)

save_figs=True
if save_figs:
    fig.savefig("AverageTravelTimes.png", dpi=200)
#     fig.savefig("AverageTravelTimes.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, tt_bimodal-wait_times, 'o-', label='bimodal')
ax.plot(x_rescaled, [tt_car for _ in x_rescaled], '--', label='car')
ax.plot(x_rescaled, [tt_unimodal-wait_time_unimodal for _ in x_rescaled], '--', label='unimodal')
ax.set_ylabel(r"$t$ (s)")
ax.set_xlabel(xlabel)
ax.legend()
# ax.set_ylim(0,1.1*tt_bimodal.max())
ax.grid()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

if save_figs:
    fig.savefig("AverageRideTimes.png", dpi=200)
#     fig.savefig("AverageRideTimes.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, tt_ratios_beyond_lcut, 'o-', label=r"$TT_\mathrm{bimodal}\,/\,TT_\mathrm{car}$")
ax.set_xlabel(xlabel)
ax.legend()
ax.grid()

if save_figs:
    fig.savefig("TTRatiosBeyondLCut.png", dpi=200)
plt.show()

# Whats happening at peak?

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, standing_fractions, 'o-', label="Standing Fraction of DRT vehicles")
ax.set_xlabel(xlabel)
ax.legend()
ax.grid()
ax.set_ylim(0,None)

save_figs=True
if save_figs:
    fig.savefig("StandingDrtFraction.png", dpi=200)
plt.show()

In [ ]:
%matplotlib inline
ph_modestats.plot.area()

plt.figure()
pkm_modestats.plot.area()
plt.show()

# Debug

In [ ]:
plt.figure()
plt.scatter(ell_list, tram_occs_theory[:,0], label='int')
plt.scatter(ell_list, tram_occs_theory[:,1], label='av')
plt.legend()

plt.show()

In [ ]:
a = np.array(["a1","a2","a3","a4","a5"])
b = np.array(["b1","b2"])
c = np.array(["c1","c2","c3","c4","c5"])
d = np.array(["d1","d2"])

for i in range(len(a)*len(b)*len(c)*len(d)):
    i1 = i%len(a)
    h1 = i//len(a)
    i2 = h1%len(b)
    h2 = h1//len(b)
    i3 = h2%len(c)
    h3 = h2//len(c)
    i4 = h3%len(b)
    print(a[i1], b[i2], c[i3], d[i4])